In [1]:
import mbuild as mb
from mbuild.lib.recipes import TiledCompound
import parmed as pmd
import numpy as np
import openmm
import warnings
from random import sample
from copy import deepcopy
warnings.simplefilter("ignore")

In [2]:
slab = mb.load("ZnS.14sqnm.pdb")
# slab.save("lammps_test/ZnS.xyz")

In [3]:
slab

<Compound 850 particles, non-periodic, 1352 bonds, id: 140655618340608>

In [12]:
slab.visualize(backend="nglview", show_ports=True)

NGLWidget()

In [6]:
slab = mb.load("ZnS.14sqnm.pdb")
for zn in slab:
    if zn.element.symbol == 'Zn' and round(zn.pos[2],3) == 0.545:
        #print(zn)
        port = mb.Port(anchor=zn, orientation=[0,0,1], separation=0.05)
        slab.add(port)

In [7]:
from random import sample

sites = sample(slab.all_ports(), k=28)

In [9]:
oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
oleic.add(mb.Port(anchor=oleic[18], separation=0.25, orientation=(oleic[18].pos - oleic[17].pos)))
oleic.visualize(show_ports=True, backend="nglview")

NGLWidget()

In [10]:
for site in sites:
    #should be coo- 
    oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
    oleic.add(mb.Port(anchor=oleic[18], separation=0.25, orientation=(oleic[18].pos - oleic[17].pos)))
    mb.force_overlap(move_this=oleic,
                     from_positions=oleic['Port[0]'],
                     to_positions=site)
    
    oleic.remove_bond((site.anchor, oleic[18]))
    slab.remove_bond((site.anchor, oleic[18]))
    slab.add(oleic, label="OLC[$]")

In [11]:
slab.visualize(show_ports=True, backend="nglview")

NGLWidget()

In [12]:
counter_slab = deepcopy(slab)

In [13]:
counter_slab.rotate(np.pi, [1,1,0])
counter_slab.translate([0,0,4])

In [14]:
counter_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
system = mb.Compound()
system.add(slab)
system.add(counter_slab)
system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
system.save("test_slabs.mol2")

OSError: test_slabs.mol2 exists; not overwriting

## Trying to get periodicity to actually work

In [ ]:
mini_slab = mb.load("ZnS.mol2")

In [ ]:
mini_slab.visualize()

In [ ]:
mini_slab

In [ ]:
for i in mini_slab:
    print(i.pos)

In [80]:
unit_cell = mb.Box([0.545027, 0.545027, 0.545027], [90, 90, 90])

In [81]:
mini_slab.box = unit_cell

In [82]:
mini_slab

<Compound 46 particles, System box: Box: Lx=0.545027, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 56 bonds, id: 140311922202704>

In [83]:
mini_slab.periodicity=(True,True,True)

In [58]:
mini_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [84]:
mini_tiles = TiledCompound(mini_slab, (2,1,1))
mini_tiles.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [85]:
mini_tiles

<Compound2-1-1 92 particles, System box: Box: Lx=1.090054, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 112 bonds, id: 140311789801536>

In [65]:
mini_tiles[0].pos

array([0., 0., 0.])

In [75]:
pos1 = [round(p,2) for p in mini_tiles[0].pos]
pos1

[0.0, 0.0, 0.0]

In [90]:
matches = []
for i,part1 in enumerate(mini_tiles):
    pos1 = [round(p,1) for p in part1.pos]
    for j,part2 in enumerate(mini_tiles):
        pos2 = [round(p,1) for p in part2.pos]
        if j != i and (part1,part2) not in matches and (part2,part1) not in matches:
            if pos1 == pos2:
                print(i,j, part1.pos)
                matches.append([part1,part2))

4 46 [0.54503 0.      0.     ]
5 47 [0.54503 0.      0.54503]
6 48 [0.54503 0.54503 0.     ]
7 49 [0.54503 0.54503 0.54503]
9 54 [0.54503    0.27250999 0.27250999]
15 74 [0.40876999 0.40876999 0.40876999]
16 65 [0.40876999 0.13626    0.13626   ]
32 64 [ 0.40876999 -0.13626    -0.13626   ]
33 66 [ 0.68128002 -0.13626     0.13626   ]
34 67 [ 0.68128002  0.13626    -0.13626   ]
35 68 [ 0.40876999 -0.13626     0.40876999]
36 69 [0.40876999 0.13626    0.68128002]
37 70 [ 0.68128002 -0.13626     0.68128002]
38 60 [0.68128002 0.13626    0.40876999]
39 71 [ 0.40876999  0.40876999 -0.13626   ]
40 72 [0.40876999 0.68128002 0.13626   ]
41 63 [0.68128002 0.40876999 0.13626   ]
42 73 [ 0.68128002  0.68128002 -0.13626   ]
43 75 [0.40876999 0.68128002 0.68128002]
44 76 [0.68128002 0.40876999 0.68128002]
45 77 [0.68128002 0.68128002 0.40876999]


In [93]:
for match in matches:
    mini_tiles.remove(match[1])

In [94]:
mini_tiles

<Compound2-1-1 71 particles, System box: Box: Lx=1.090054, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 84 bonds, id: 140311789801536>

In [95]:
mini_tiles.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
for port in mini_tiles.all_ports()

In [96]:
mini_tiles.all_ports()

[<Port, anchor: 'ZN', labels: , id: 140311788749776>,
 <Port, anchor: 'ZN', labels: , id: 140311788749680>,
 <Port, anchor: 'ZN', labels: , id: 140311786503616>,
 <Port, anchor: 'ZN', labels: , id: 140311788751408>,
 <Port, anchor: 'ZN', labels: , id: 140311786555616>,
 <Port, anchor: 'ZN', labels: , id: 140311788749392>,
 <Port, anchor: 'ZN', labels: , id: 140311788814448>,
 <Port, anchor: 'ZN', labels: , id: 140311788751888>]

In [78]:
mini_tiles.save("min_test.xyz")

In [79]:
mini_tiles.energy_minimize()

*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested

In [2]:
slab = 
for zn in slab:
    if zn.element.symbol == 'Zn' and round(zn.pos[2],3) == 0.545:
        print(zn)
        port = mb.Port(anchor=zn, orientation=[0,0,1], separation=0.05)
        slab.add(port)

/home/rramji/.conda/envs/openmm/lib/python3.9/site-packages/mbuild/conversion.py:481: UserWarning: Using parmed reader. Bonds may be inferred from inter-particle distances and standard residue templates. Please check that the bonds in mb.Compound are accurate
  warn(


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
import mbuild as mb
from mbuild.lib.surfaces import Betacristobalite

surface = Betacristobalite()
tiled_surface = mb.lib.recipes.TiledCompound(surface, n_tiles=(2, 1, 1))


#######################################################
The code at conversion.py:429 requires the gmso package
#######################################################


DelayImportError: The code at conversion.py:429 requires the gmso package